<a href="https://colab.research.google.com/github/Charles1A/animated_scatter_plot/blob/main/Ndaq-100-animtd-scatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf

import scipy
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta, date
import time

In [14]:
print(f"pandas version: {pd.__version__}\n"\
f"numpy version: {np.__version__}\n"\
f"yfinance version: {yf.__version__}\n")

print("plotly version: ")
!pip show plotly | grep Version

pandas version: 2.2.2
numpy version: 1.26.4
yfinance version: 0.2.49

plotly version: 
Version: 5.24.1


In [ ]:
# Load the Nasdaq 100 csv:

df = pd.read_csv('/Nasdaq-100.csv')
df.head(3)

,SYMBOL,NAME,PRICE,CHANGE,%CHANGE
0,AMD,Advanced Micro Devices Inc,98.01,0.13,0.13
1,ADBE,Adobe Inc.,385.37,3.47,0.91
2,ABNB,Airbnb Inc,124.4,2.12,1.73


In [ ]:
# df.info()

In [ ]:
# df.iloc[: ,0]

In [ ]:
# Create a list of tickers from the Nasdaq 100 csv:

ticker_list = df.iloc[: ,0].to_list()
ticker_list[0:5]

['AMD', 'ADBE', 'ABNB', 'ALGN', 'AMZN']

In [ ]:
# Join the tickers into a single string for the yfinance API:

ticker_string = ' '.join(ticker_list)
print(ticker_string[0:20])

AMD ADBE A


In [ ]:
def qrtr_func(sdate, edate):

    stock_df = yf.download(f'{ticker_string}', # Make call to the yfinance API
        start=sdate, end=edate)

    time.sleep(4) # 4-second pause between API calls; comment out if making just one call

    Adjclose = stock_df.loc[:, 'Adj Close'] # Use adjusted close for computations

    Adjclose = Adjclose.rename_axis(index=None, columns=None)

    Qreturns = ((Adjclose.iloc[-1, :] - Adjclose.iloc[0, :])/Adjclose.iloc[0, :]).to_frame() # Compute returns for the quarter

    returns1d = Adjclose.pct_change() # Compute the 1-day returns for std dev
    returns1d.dropna(inplace=True)

    Qstd = returns1d.dropna().std().to_frame() # Compute std dev

    qrtr_df = pd.concat([Qreturns, Qstd], axis=1) # Concatenate the two dfs
    qrtr_df.columns = ["Pct Returns", "Std dev"]

    qrtr_df['Pct Returns'] = qrtr_df['Pct Returns']*100

    sdate_ = pd.to_datetime(sdate)

    Q = sdate_.to_period('Q')

    qrtr_df['quarter'] = Q

    qrtr_df['quarter'] = qrtr_df['quarter'].astype(str)

    return qrtr_df

# Call the function with appropriate start and end dates to capture data for the quarter;
# Note: edate must be the first date of the next quarter

Y2022Q2 = qrtr_func('2022-04-01', '2022-07-01')
Y2022Q3 = qrtr_func('2022-07-01', '2022-10-01')
Y2022Q4 = qrtr_func('2022-10-01', '2023-01-01')
Y2023Q1 = qrtr_func('2023-01-01', '2023-04-01')

[*********************100%***********************]  101 of 101 completed
[*********************100%***********************]  101 of 101 completed
[*********************100%***********************]  101 of 101 completed


In [ ]:
# Create a color category to highlight certain markers on the scatter plot (animated or still):

dflist = [Y2022Q2, Y2022Q3, Y2022Q4, Y2023Q1]

def tracker(dflist):

    for i in range(len(dflist)):

        df = dflist[i].reset_index(inplace=False)

        df['Tracking'] = df.iloc[: , 0].apply(lambda x: 'magenta' if x == 'TSLA' or x == 'LCID' or x == 'NVDA' or x == 'META' or x == 'ALGN' else 'steelblue')

        dflist[i] = df

    return dflist

tracker(dflist)

[    index  Pct Returns   Std dev quarter   Tracking
 0    AAPL   -21.449784  0.025790  2022Q2  steelblue
 1    ABNB   -48.529498  0.044561  2022Q2  steelblue
 2    ADBE   -20.107380  0.027728  2022Q2  steelblue
 3     ADI   -11.242042  0.022863  2022Q2  steelblue
 4     ADP    -9.619842  0.019553  2022Q2  steelblue
 ..    ...          ...       ...     ...        ...
 96    WBD   -46.362910  0.040649  2022Q2  steelblue
 97   WDAY   -41.335684  0.033615  2022Q2  steelblue
 98    XEL    -2.036367  0.013376  2022Q2  steelblue
 99     ZM    -8.515502  0.044547  2022Q2  steelblue
 100    ZS   -39.275419  0.054524  2022Q2  steelblue
 
 [101 rows x 5 columns],
     index  Pct Returns   Std dev quarter   Tracking
 0    AAPL    -0.387272  0.019300  2022Q3  steelblue
 1    ABNB    14.910838  0.030509  2022Q3  steelblue
 2    ADBE   -25.314806  0.030104  2022Q3  steelblue
 3     ADI    -2.028811  0.019512  2022Q3  steelblue
 4     ADP     6.536322  0.016101  2022Q3  steelblue
 ..    ...         

In [ ]:
# Concatenate the dfs:

concat_df = pd.concat(dflist, axis=0)
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404 entries, 0 to 100
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype        
---  ------       --------------  -----        
 0   index        404 non-null    object       
 1   Pct Returns  404 non-null    float64      
 2   Std dev      404 non-null    float64      
 3   quarter      404 non-null    period[Q-DEC]
 4   Tracking     404 non-null    object       
dtypes: float64(2), object(2), period[Q-DEC](1)
memory usage: 18.9+ KB


In [ ]:
# Create animated scatter plot via Plotly Express:

fig = px.scatter(concat_df,
                x='Std dev',
                y='Pct Returns',
                color='Tracking',
                hover_name='index',
                 log_x=False,
                 animation_frame='quarter',
                 range_x=[0, 0.09],
                 range_y=[-60,110],
                 text='index'
                )

fig.update_traces(textposition='top center')

fig.update_layout(
    showlegend=False,
    height=600,
    width=700,
    title_text='Nasdaq 100',
    title_x=0.5)

fig.update_traces(marker=dict(size=12,
                              line=dict(width=2, color='white')))

fig.add_hrect(y0=0, y1=-60,
              line_width=0,
              fillcolor="red", opacity=0.1,
                annotation_text="[NEGATIVE RETURN REGION]",
                annotation_position="top right",
                annotation_font_size=10,
                annotation_font_color="black")

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 2750
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1500

fig.show()
# fig.write_html("") # Save the output